# Introduction 2

## Bonus: Nonlinear Relationships

Here is some R code showing some nonlinear relationships that cannot be described using a simple Pearson's correlation.

First of all, these examples will all be using `tibble` and `ggplot2`, so make sure these libraries are loaded!

In [ ]:
# load libraries used in the examples
library(tibble)
library(ggplot2)

# set the ggplot2 theme
theme_set(theme_classic())

### Example 1: Sine Wave with some noise

Here we generate 250 observations from a Sine wave, with normally distributed noise of SD=.25

In [ ]:
example_1 <- tibble(
  # randomly sample x values from 0 to 25
  x = runif(n=250, min=0, max=25),
  # get the sine of x
  s = sin(x),
  # generate random noise of sd=.25
  e = rnorm(n=250, mean=0, sd=0.25),
  # add the sine wave to the noise to get the observations
  y = s + e
)

# plot the observations
ggplot(example_1, aes(x, y)) +
  geom_point()

But as you can see, the Pearson's correlation is very close to zero - why do you think that is?

In [ ]:
# get the Pearson's correlation
cor(example_1$x, example_1$y)

We can also visualise the linear trend that the Pearson's correlation is evaluating - clearly, the linear model is inappropriate!

In [ ]:
# superimpose the simple linear model
ggplot(example_1, aes(x, y)) +
  geom_point() +
  geom_smooth(method="lm", formula=y~x)